# Project 3: Logistic Regression

##  Preparation and Overview (30 points total)

### [20 points] Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). 

# Business Understanding

This data set is from The Department of Transportation’s Bureau of Transportation Statistics regarding the On Time Performance of domestic flights flying from the DFW airport from January to March of this year. Since the volume of the original data is so large, we decided to only look at flights from DFW to ORD (Chicago O'Hare) for our model. 

When booking flights for a trip, there are often many different airlines and times of day to choose from. Our classification task is to predict the departure delay group (how long the flight is delayed divided into 14 groups of 15 minute intervals ranging from <-15 minutes (flight is more than 15 minutes early) to >= 180 minutes) for each flight based on the Month, Day of Flight, Day of the Week, Departure Time, Airline. The end goal of analysis on our dataset is to be able to understand for an origin and a destination, in our case DFW to ORD, when is the best time to fly and with what airline in order to minimize departure delays when booking a trip. If our model is successful, it could be trained with other origin and destination cities.

## Who would benefit

According to air travel intelligence company, OAG, from April 2017 to March 2018 DFW to ORD was DFW’s second highest grossing route, bringing in $358.4 million in revenue. Thus, it is in the airlines best interest to minimize departure delay and maximize customer satisfaction on this route. Although small delays are inevitable, airlines that fly from DFW could use this analysis to help them see how they stack up to their competitors and to help them better schedule their employees in order to account for probable delays. For example, during times when there is a high likelihood of a long delay, they could have more gate and travel agent staff available.


Additionally, businesses who send employees on business trips and people taking personal trips flying from DFW to ORD would benefit from this analysis. If a flight for a business trip gets delayed or cancelled, the company loses money as hours of the client or employee's time are wasted as a result. Additionally, many people get a very limited time for vacation and personal trips. A long delay or cancellation can cause them to lose valuable time at their destination or with their families. Using this analysis a person or company can try to schedule their flights to minimize likelihood of experiencing delays when flying from DFW. If a person has to fly during a time or with an airline with a likelihood of a long delay, they can use this analysis to help them plan accordingly and be sure to build in flexibility due to delays when planning their schedules.

## Model Performance
In order to be considered a useful model, the model will need to be able to accurately predict 90% of the entire dataset. Our model will not be perfect because delay time can be affected by a myriad of factors such as mechanical factors, weather related factors, crew related factors, and the fact that a plane does not just magically appear in an airport but travels from a different destination where it could have also been affected by delays. With the given data our model is unable to account for all of these factors.

Our model would most likely be deployed for internal analysis by the airlines and used for offline analysis by  businesses, and customers planning a trip.

For businesses and people using this model to try and plan their schedules when taking a personal or business trip a number of false negatives (where a flight is predicted to have a large delay but is not delayed) would not be too much of an issue, but a large number of false positives (where a flight is predicted to be early or have little or no delay but is delayed) would be problematic. Although some delays are unavoidable, it is not ethical or useful to the customer to have a large number of false positives. Because of this, our model would have to have a relative rate of around 1% of false positives.

### [5 points] (mostly the same processes as from previous labs) Define and prepare your class variables. 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline
import missingno as mn
import pandas as pd
import numpy as np

df = pd.read_csv('DallasToChicago.csv') # read in the csv file
print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)

#Remove attributes that are not useful for us
for col in ['TailNum','FlightNum','OriginAirportID',
           'OriginCityName','OriginState','OriginStateName','DestAirportID','DestCityName','DestState','DestStateName','CRSDepTime',
           'DepDelayMinutes','TaxiIn','TaxiOut','CRSArrTime','ArrTime','ArrDelay','ArrDelayMinutes','ArrDelayGroup','ATimeBlk','CancellationReason',
            'Diverted', 'AirTime','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','FirstDepTime1','FirstDepTime2',
            'FirstDepTime','TotalAddGTime','LongestAddGTime','DivAirportLandings','DivReachedDest','DivActualElapsedTime','DivArrDelay','DivDistance',
           'CRSElapsedTime','Flights','Cancelled','Unnamed: 0', 'Distance', 'DistGroup', 'ActualElapsedTime']:
    if col in df:
        del df[col]

print(df.dtypes)

In [ ]:
print(df.info(verbose=True, null_counts=True))


In [ ]:
#Figure out what data is missing
df.isnull().sum()

In [ ]:
mn.matrix(df)

In [ ]:
#Remove the missing data
df.dropna(inplace=True)
df.count()

In [ ]:
print(df.shape)
df.dtypes

Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).
Initial Dataset:
* month (the day of the week of the flight) -----------------------> ordinal
* dayofmonth (the day of the week of the flight) -----------------------> ordinal
* dayofweek (the day of the week of the flight) -----------------------> ordinal
* airline (the airline of the flight)--------------------------------> nominal
* Origin (origin airport code of the flight)-------------------------> nominal
* Dest (destination airport code of the flight)----------------------> nominal
* DepTime (departure time of the flight)-----------------------------> ratio
* DepDelay (delay of the flight departure in minutes)----------------> interval
* DepDelayGroup (delay of the flight departure grouped by minutes)---> ordinal
* DTimeBlk (delay of the flight grouped by hours)--------------------> ordinal
* ActualElapsedTime (flight time in the air)-------------------------> ratio
* Distance (distance the flight travelled)---------------------------> interval
* DistGroup (distance the flight travelled grouped by miles----------> ordinal

Final Dataset: 
* month (the day of the week of the flight) -----------------------> int
* dayofmonth (the day of the week of the flight) -----------------------> int
* dayofweek (the day of the week of the flight) -----------------------> int
* airline (the airline of the flight)--------------------------------> one-hot encoded
* DepTime (departure time of the flight in seconds)-----------------------------> float
* DepDelayGroup (delay of the flight departure grouped by minutes)---> int



In [ ]:
#see explained variance
import numpy as np

def plot_explained_variance(pca):
    import plotly
    from plotly.graph_objs import Bar, Line
    from plotly.graph_objs import Scatter, Layout
    from plotly.graph_objs.scatter import Marker
    from plotly.graph_objs.layout import XAxis, YAxis
    plotly.offline.init_notebook_mode() # run at the start of every notebook

    explained_var = pca.explained_variance_ratio_
    cum_var_exp = np.cumsum(explained_var)

    plotly.offline.iplot({
        "data": [Bar(y=explained_var, name='individual explained variance'),
                 Scatter(y=cum_var_exp, name='cumulative explained variance')
            ],
        "layout": Layout(xaxis=XAxis(title='Principal components'), yaxis=YAxis(title='Explained variance ratio'))
    })

features = ['Year','Quarter','Month' ,'DayofMonth','DayOfWeek','DepDelay']



In [ ]:
from sklearn.preprocessing import StandardScaler

# Separating out the features
x = df.loc[:, features].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_pca = pca.fit(x)
plot_explained_variance(pca)


#### [5 points] Divide you data into training and testing data using an 80% training and 20% testing split. 

Use the cross validation modules that are part of scikit-learn. Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

In [ ]:
import datetime
import time

def time_converter(t):
    x = time.strptime(t.split(',')[0],'%H:%M')
    return int(datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())



df["RealDepTime"] = df.apply( lambda row: time_converter(row.DepTime), axis=1)


cleaned_df = df[['Month' ,'airline','DayofMonth','DayOfWeek', 'DepDelayGroup','RealDepTime']]
columns = cleaned_df # Declare the columns names
y = columns

#one hot encode airline
temp_df = pd.get_dummies(cleaned_df.airline,prefix='airline')
cleaned_df = pd.concat((cleaned_df,temp_df),axis=1)

In [ ]:
if 'airline' in cleaned_df:
    del cleaned_df['airline'] # get rid of the original category as it is now one-hot encoded

In [ ]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
cleaned_df['DepDelayGroupA'] = number.fit_transform(cleaned_df['DepDelayGroup'].astype('str'))
#cleaned_df.head()

print(cleaned_df['DepDelayGroupA'].nunique())
print(df['DepDelayGroup'].nunique())

#show the value that coorsponds to original groups
comp_df = cleaned_df[['DepDelayGroup' ,'DepDelayGroupA']]
print(comp_df.drop_duplicates())


In [ ]:
if 'DepDelayGroup' in cleaned_df:
    del cleaned_df['DepDelayGroup'] # get rid of the original category as it is now one-hot encoded
cleaned_df.head()

In [ ]:
# create training and testing vars
##y_train, y_test, X_train, X_test = train_test_split(cleaned_df, y, test_size=0.2)
#remove label from training dataset
#X_train = X_train.drop(['DepDelayGroup'], axis=1)
#print(X_train.shape)
#print(X_test.shape)
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'DepDelayGroupA' in cleaned_df:
    y = cleaned_df['DepDelayGroupA'].values # get the labels we want
    del cleaned_df['DepDelayGroupA'] # get rid of the class label
    norm_features = ['Month','DayofMonth','DayOfWeek','RealDepTime' ]
    cleaned_df[norm_features] = (cleaned_df[norm_features]-cleaned_df[norm_features].mean()) / cleaned_df[norm_features].std()
    X = cleaned_df.values # use everything else to predict!

In [ ]:
num_cv_iterations = 1
num_instances = len(y)
cv_object = ShuffleSplit(
                         n_splits=num_cv_iterations,
                         test_size  = 0.2)
print( cv_object.split(X,y))
# the indices are the rows used for training and testing in each iteration

X_train, y_train, X_test, y_test = ([] for i in range(4))

for train_indices, test_indices in cv_object.split(X,y):
    # I will create new variables here so that it is more obvious what
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
num_cv_iterations = 1
num_instances = len(y)
cv_object = ShuffleSplit(
                         n_splits=num_cv_iterations,
                         test_size  = 0.2)
print( cv_object.split(X,y))

In [ ]:
X_train, y_train, X_test, y_test = ([] for i in range(4))

for train_indices, test_indices in cv_object.split(X,y):
    # I will create new variables here so that it is more obvious what
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]

    X_test = X[test_indices]
    y_test = y[test_indices]

    X_test = X[test_indices]
    y_test = y[test_indices]

In [ ]:
#explaniaiton of training split
# divide into testing and training
# a ration of 80:20 would be great for our dataset because it is neither too small nor too bigself.
# an average dataset like this one would not need more than 20% of data as testing because 362 is already enough to capture most of the variation
#  But also since our data is not computationally expensive the test data does not need to be less than 20% either


### Modeling (50 points total)

#### The implementation of logistic regression must be written only from the examples given to you by the instructor. No credit will be assigned to teams that copy implementations from another source, regardless of if the code is properly cited. 

#### [20 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. 

Use object oriented conventions identical to scikit-learn. You should start with the template developed by the instructor in the course. You should add the following functionality to the logistic regression classifier:Ability to choose optimization technique when class is instantiated: either steepest descent, stochastic gradient descent, or Newton's method.  Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1 and L2 regularization). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.  

In [ ]:
# BLR
#inherit from base class
# from last time, our logistic regression algorithm is given by (including everything we previously had):
from sklearn import metrics as mt
from scipy.special import expit
from numpy.linalg import pinv
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001, optChoice='steepest', reg_choice = "o"):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.optChoice = optChoice
        self.reg_choice = reg_choice
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'

    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))


    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1

    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction


    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate

    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate


In [ ]:
#Vector BinaryLogisticRegression
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))

    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        if self.optChoice == 'steepest':
            ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
            gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)


            l_choice = self.reg_choice
            if l_choice == "o":
                gradient += gradient.reshape(self.w_.shape)
            elif l_choice == "l1":
                gradient[1:] += -np.sin(self.w_[1:]) * self.C
            elif l_choice == "l2":
                gradient[1:] += -2 * self.w_[1:] * self.C
            elif l_choice == "both":
                gradient[1:] += (-np.sin(self.w_[1:]) + (-2 * self.w_[1:])) * self.C

            return gradient
        elif self.optChoice == 'stochastic':
            # stochastic gradient calculation
            idx = int(np.random.rand()*len(y)) # grab random instance
            ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
            gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)


            l_choice = self.reg_choice
            if l_choice == "o":
                gradient += gradient.reshape(self.w_.shape)
            elif l_choice == "l1":
                gradient[1:] += -np.sin(self.w_[1:]) * self.C
            elif l_choice == "l2":
                gradient[1:] += -2 * self.w_[1:] * self.C
            elif l_choice == "both":
                gradient[1:] += (-np.sin(self.w_[1:]) + (-2 * self.w_[1:])) * self.C


            return gradient
        elif self.optChoice == 'newtonHessian':
            g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
            hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian
            ydiff = y-g # get y difference
            gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)

            l_choice = self.reg_choice
            if l_choice == "o":
                gradient += gradient.reshape(self.w_.shape)
            elif l_choice == "l1":
                gradient[1:] += -np.sin(self.w_[1:]) * self.C
            elif l_choice == "l2":
                gradient[1:] += -2 * self.w_[1:] * self.C
            elif l_choice == "both":
                gradient[1:] += (-np.sin(self.w_[1:]) + (-2 * self.w_[1:])) * self.C

            return pinv(hessian) @ gradient


In [ ]:
#Logistic Regression
class LogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001, optChoice='steepest', reg_choice="o"):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.optChoice = optChoice
        self.reg_choice = reg_choice
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers

        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,self.iters,self.C,self.optChoice, self.reg_choice)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier

        return np.hstack(probs) # make into single matrix

    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row


In [ ]:
import numpy as np
def getCArray(beginC, endC, stepSize):
    cArr = []
    for i in np.arange(beginC, endC, stepSize).tolist():
        cArr.append(i)
    return cArr


In [ ]:
def snoopReg(beginC, endC, stepSize, X_train, y_train,X_test,y_test, regression):
    accuracyArr = []
    for i in np.arange(beginC, endC, stepSize).tolist():
        #Choose the optimization and the L term
        if (regression == "lr_steep0"):
            lr = LogisticRegression(eta=0.1, C = i)
        elif (regression == "lr_steep1"):
            lr = LogisticRegression(eta=0.1,C = i,reg_choice = "l1")
        elif (regression == "lr_steep2"):
            lr = LogisticRegression(eta=0.1, C = i, reg_choice = "l2")
        elif (regression == "lr_steepb"):
            lr = LogisticRegression(eta=0.1, C= i, reg_choice = "both")
        elif (regression == "lr_scho0"):
            lr = LogisticRegression(eta=0.1,iterations=1500,C = i, optChoice = 'stochastic')
        elif (regression == "lr_scho1"):
            lr = LogisticRegression(eta=0.1,iterations=1500, C = i, optChoice = 'stochastic',reg_choice = "l1")
        elif (regression == "lr_scho2"):
            lr = LogisticRegression(eta=0.1,iterations=1500, C = i, optChoice = 'stochastic',reg_choice = "l2")
        elif (regression == "lr_schob"):
            lr = LogisticRegression(eta=0.1,iterations=1500, C = i, optChoice = 'stochastic',reg_choice = "both")
        elif (regression == "lr_nh0"):
            lr = LogisticRegression(eta=0.1,iterations=1, C = i, optChoice = 'newtonHessian')
        elif (regression == "lr_nh1"):
            lr = LogisticRegression(eta=0.1,iterations=1, C = i, optChoice = 'newtonHessian',reg_choice = "l1")
        elif (regression == "lr_nh2"):
            lr = LogisticRegression(eta=0.1,iterations=1, C = i, optChoice = 'newtonHessian',reg_choice = "l2")
        elif (regression == "lr_nhb"):
            lr = LogisticRegression(eta=0.1,iterations=1, C = i, optChoice = 'newtonHessian',reg_choice = "both")
        lr.fit(X_train,y_train)  # train object
        y_hat = lr.predict(X_test) # get test set precitions
        acc = mt.accuracy_score(y_test,y_hat)
        accuracyArr.append(acc)
    return accuracyArr


#### [15 points] Train your classifier to achieve good generalization performance.

 That is, adjust the optimization technique and the value of the regularization term "C" to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?

In [ ]:
regListName = ["Steepest-Orig :", "Steepest-1 :", "Steepest-2 :", "Steepest-B :", "Stochastic-Orig: ", "Stochastic-1: ", "Stochastic-2: ", "Stochastic-B: ", "NewtonHessian-Orig: ", "NewtonHessian-1: ", "NewtonHessian-2: ", "NewtonHessian-B: "]
regList = ["lr_steep0", "lr_steep1", "lr_steep2", "lr_steepb", "lr_scho0", "lr_scho1", "lr_scho2", "lr_schob", "lr_nh0", "lr_nh1", "lr_nh2", "lr_nhb"]
cList = [0.001,1,0.01]
i = 0
bestC = []
for r in regList:
    regArr = snoopReg(beginC = cList[0], endC = cList[1], stepSize = cList[2], X_train = X_train, y_train = y_train, X_test = X_test,y_test = y_test, regression = r)
    cArr = getCArray(beginC = cList[0], endC = cList[1], stepSize = cList[2])
    print(regListName[i])
    print("max accuracy: " , max(regArr))
    c_value_index = regArr.index(max(regArr))
    bestC.append(cArr[c_value_index])
    print("c value: ", cArr[c_value_index])
    i+=1

#### [15 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. Discuss the results. 
   

In [ ]:
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.metrics import accuracy_score
lr_sk = SKLogisticRegression() # all params default

lr_sk.fit(X,y)
yhat = lr_sk.predict(X)
print('Accuracy of: ',accuracy_score(y,yhat))

#### Deployment (10 points total)
        

Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

#### Exceptional Work (10 points total)
        

You have free reign to provide additional analyses. One idea: Update the code to use either "one-versus-all" or "one-versus-one" extensions of binary to multi-class classification. 
        One idea (required for 7000 level students): Implement an optimization technique for logistic regression using mean square error as your objective function (instead of binary entropy). Your solution should be able to solve the binary logistic regression problem in one gradient update step. 